# Testing notebook

In [30]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor

## We need some testing data

In [46]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

,numeric_variable,object_variable,column with soo000oo000oo000oo000oo long name
0,15.384871,2.0,13.116582
1,14.351146,9.0,5.256467
2,12.617547,9.0,14.558294
3,16.714980,3.0,8.296990
4,NaN,NaN,-0.249980
...,...,...,...
995,8.897679,7.0,8.587763
996,3.326277,5.0,13.691772
997,4.948288,2.0,9.136339
998,8.080602,4.0,10.802152


## Computions funcitons

In [32]:
from computions import *

### get_describe_nominal function test

In [49]:
get_describe_nominal(test_frame.iloc[:,1], Y)
temp = na_containts_frame.iloc[:,1].fillna('empty')
get_describe_nominal(temp, Y)

,object_variable,1.0,1.0%,2.0,2.0%,0.0,0.0%
2.0,109,29,26.605505,0.0,0.000000,80.0,73.394495
7.0,104,91,87.500000,12.0,11.538462,1.0,0.961538
1.0,102,7,6.862745,0.0,0.000000,95.0,93.137255
3.0,99,46,46.464646,0.0,0.000000,53.0,53.535354
8.0,97,83,85.567010,14.0,14.432990,0.0,0.000000
4.0,96,71,73.958333,0.0,0.000000,25.0,26.041667
9.0,94,50,53.191489,44.0,46.808511,0.0,0.000000
6.0,89,85,95.505618,2.0,2.247191,2.0,2.247191
5.0,85,68,80.000000,3.0,3.529412,14.0,16.470588
0.0,57,2,3.508772,0.0,0.000000,55.0,96.491228


### AUC computing test

AUC numeric - simple fucntion

In [53]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5581294503365741, 2.0: 0.7590790679475694, 0.0: 0.32240474006941067}
{'category 1': 0.5581294503365741, 'category 2': 0.7590790679475694, 'cetegory 0': 0.32240474006941067}
{'category 1': 0.5576121267365508, 'category 2': 0.7624023142351367, 'cetegory 0': 0.3224880293281461}
{1.0: 0.6775952599305892, 0.0: 0.32240474006941067}


AUC nominal

In [58]:
def get_AUC_nominal(column, y_col, descr_table = None):
    '''Returns AUC for nominal predictor. Levels will be
    displayed in order of increasing part of predicted level.
    Nas in column will be ignored, should be processed before '''
    # inputs:
    # column - pandas.Series predictors column
    # y_col - pandas.Series predicted column
    # descr_table - pandas.DataFrame (optional), out of get_describe_nominal
    #               function for predictor. Will be computed automaticaly by default
    # outputs:
    # dictionary which contains result {<y_levels>:<AUC for this levels>}
    if descr_table is None:
        descr_table = get_describe_nominal(column, y_col)
    
    # in case column has only one level need return 0.5
    # for all y levels - predictor are useless
    if descr_table.shape[0] <= 1:
        return {level : 0.5 for level in y_col.unique()}

    result = {}

    for level in y_col.unique():
        temp_binary_column = y_col.apply(lambda x: 0 if x != level else 1)
        level_distr = descr_table[str(level) + "%"].sort_values()
        numbers = column.replace({n:i for i,n in enumerate(level_distr.index)})
        result[level] = roc_auc_score(temp_binary_column, numbers/sum(numbers))

    return result

non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

{'category 1': 0.8582052352371419, 'category 2': 0.9232642428361083, 'cetegory 0': 0.9437993382604161}
AUC bynary
{1.0: 0.943799338260416, 0.0: 0.9437993382604161}
AUC with emptys
{'category 1': 0.7284418730271265, 'category 2': 0.9219182843720705, 'cetegory 0': 0.057538461538461524}


## Creating a writer to excel

In [5]:
xl_writer = pd.ExcelWriter("test_result/test.xlsx",engine='xlsxwriter')

In [6]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

my_cpp.write_to_book(xl_writer)

In [11]:
predictos_data = my_cpp.get_predictors_data()
predictos_data[0]['describe_table']

,Value:,Part %:
count,1000.000000,NaN
mean,10.063304,NaN
std,4.765109,NaN
min,-4.154555,NaN
25%,6.706783,NaN
50%,9.870845,NaN
75%,13.436595,NaN
max,26.395762,NaN
Emptys,0.000000,0.0


## Saving the results

In [7]:
xl_writer.close()